In [7]:
import os 
import math
from datetime import timedelta
import seaborn as sns
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm 
from Modele_Lasso_IC import previsions_Lasso
os.getcwd()
dir_path= "/Users/h2jw/Documents/GitHub/StatApp_2020/"
os.chdir(dir_path)

In [8]:
!ls

Bibliographie       Modélisation        README.md
Data                Preparation reunion Visualisation


In [ ]:
dateDebMod = pd.to_datetime("2008-01-01")
dateFinMod = pd.to_datetime("2015-12-31")

path = "/Users/h2jw/Documents/GitHub/StatApp_2020/"
#horizonsPrev = [7, 31+29+31, 365]  # (en jours) [365] 
ic = 0.95   # Seuil de l'intervalle de confiance souhaité
            



if __name__ == '__main__':
    
    database = pd.read_csv("/Users/h2jw/Downloads/database_sieges.csv",low_memory=False,decimal=',')
    database = database.astype({'Date': 'datetime64[ns]','PAX_FQM':'float','Sièges Corrections_ICI':'float','Coeff_Rempl':'float','Coeff_Rempl_FQM':'float'})
    database = database.groupby(['Date','Faisceau','ArrDep']).agg({'PAX':'sum','PAX_FQM':'sum','Sièges Corrections_ICI':'sum','Coeff_Rempl':'mean','Coeff_Rempl_FQM':'mean'}).reset_index()

    Calendrier = pd.read_csv(path+"Data/Calendrier/Calendrier.csv", dayfirst = True , sep = ';' , parse_dates = ['Date'])

    histoMod = database[(database['Date']>=dateDebMod) & (database['Date']<=dateFinMod)]
    # histoMod.to_csv("HistoMod.csv")
    

    hPrev=91

    # on va ajouter les prévisions à l'historique précédent + sur la période de prévisions
    histoPrev = database[(database['Date']>=dateDebMod) & (database['Date']<=dateFinMod+timedelta(days = hPrev))]
        # ( sans historique précédent : histoPrev = database[(database['Date']>dateFinMod) & (database['Date']<=dateFinMod+timedelta(days = hPrev))]   )

    Prev_Lasso = pd.DataFrame()


    for faisceau in ['National', 'Schengen', 'Autre UE', 'International', 'Dom Tom'] :
        for mvt in ['Départ']: #, 'Arrivée']: 

            histoMod_2 = histoMod[(histoMod['Faisceau']==faisceau) & (histoMod['ArrDep']==mvt)]

           # Modèle Lasso : 
            prev_Lasso = previsions_Lasso (histoMod_2, Calendrier, dateDebMod, dateFinMod, hPrev, ic)
            Prev_Lasso = pd.concat([Prev_Lasso,prev_Lasso],ignore_index=True)


    # Ajout des prévisions des différents modèles à histoPrev           
    histoPrev = pd.concat([histoPrev.set_index(['Date','Faisceau','ArrDep']),
                           Prev_Lasso.set_index(['Date','Faisceau','ArrDep'])],axis=1)
    histoPrev = histoPrev.reset_index()

    histoPrev.to_csv("Previsions_Lasso"+str(hPrev)+"j.csv")



  1%|          | 6/1000 [00:33<1:32:06,  5.56s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4951063.4946632385, tolerance: 4029852.2275170153
  model = cd_fast.enet_coordinate_descent(
  8%|▊         | 78/1000 [06:27<1:10:30,  4.59s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3746153.9369831085, tolerance: 3099765.9782182286
  model = cd_fast.enet_coordinate_descent_gram(
  9%|▉         | 94/1000 [07:48<1:05:52,  4.36s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 594592

 44%|████▎     | 436/1000 [31:49<34:11,  3.64s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3630999.22928524, tolerance: 3186115.6992955054
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5854942.921602249, tolerance: 3962972.8435028414
  model = cd_fast.enet_coordinate_descent(
 45%|████▌     | 454/1000 [32:48<32:24,  3.56s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4324699.281803131, tolerance: 3967198.479956933
  model = cd_

 36%|███▌      | 361/1000 [10:51<16:09,  1.52s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2364257.99860096, tolerance: 2131620.9379811217
  model = cd_fast.enet_coordinate_descent_gram(
 44%|████▍     | 445/1000 [13:08<15:11,  1.64s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2791847.253624916, tolerance: 2130297.045733362
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3706928.4230651855, tolerance: 2197561.9348929795
  model

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20710.509143292904, tolerance: 20308.630035945105
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21557.483839273453, tolerance: 20308.630035945105
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23250.218716442585, tolerance: 20419.40208401948
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sk

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25458.126297295094, tolerance: 20745.639987073926
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27558.497759997845, tolerance: 20745.639987073926
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26342.951953202486, tolerance: 20745.639987073926
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/s

...

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31477.87975782156, tolerance: 20137.28756573705
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 30887.514001995325, tolerance: 20816.81697039823
  model = cd_fast.enet_coordinate_descent_gram(
 98%|█████████▊| 985/1000 [1:03:01<01:11,  4.74s/it]/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22975.362137705088, tolerance: 20418.383778574593
  model = cd_fast.enet_coordinate_descent_gram(
 99%|█

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25195.112962901592, tolerance: 20490.534870916337
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31753.491470962763, tolerance: 20490.534870916337
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29591.028087943792, tolerance: 20490.534870916337
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/s

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93446.22278127074, tolerance: 20768.333915626383
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 133448.9057584107, tolerance: 20768.333915626383
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 136267.10576853156, tolerance: 20768.333915626383
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/skl

/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 43964.14726239443, tolerance: 21200.394543426297
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 57455.197903096676, tolerance: 21596.696142300887
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 71933.6337954402, tolerance: 21596.696142300887
  model = cd_fast.enet_coordinate_descent_gram(
/Users/h2jw/opt/anaconda3/lib/python3.8/site-packages/skle

 84%|████████▎ | 835/1000 [26:07<04:16,  1.55s/it]

In [ ]:
Prev_Lasso.to_csv('prevlasso.csv')

In [17]:
!ls Modélisation/Previsions

Previsions_365j.csv Previsions_7j.csv   Previsions_91j.csv


In [21]:
df=pd.read_csv('/Users/h2jw/Documents/GitHub/StatApp_2020/Modélisation/Previsions/Previsions_365j.csv',low_memory=False,decimal=',')

In [24]:
df['Date']=df['Date'].astype('datetime64[ns]')
Prev_Lasso['Date']=Prev_Lasso['Date'].astype('datetime64[ns]')

In [32]:
df= df.sort_values(by='Date')
Prev_Lasso = Prev_Lasso.sort_values(by='Date').set_index('Date')

In [28]:
len(df), len(Prev_Lasso)

(32695, 35)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='darkgrid')
fig, ax = plt.subplots(figsize=(12,6))


Prev_Lasso['PAX_Lasso'].plot(ax = ax,legend=True,style='o-')
Prev_Lasso['IC95_low_LASSO'].plot(ax=ax,legend=True,style='o--')
Prev_Lasso['IC95_up_LASSO'].plot(ax=ax,legend=True,style='o--')
